<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/test/notebooks/NeuralBeagle_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install -qU transformers accelerate

In [18]:
from transformers import AutoTokenizer
import transformers
import torch

In [19]:
function = """
def checkGPU(tensorflow):
    if tensorflow == True:
        import tensorflow as tf
        print("Number of GPUs available with tensorflow:", len(tf.config.list_physical_devices('GPU')))
    else:
        import torch
        print('Checking if the GPU is available with PyTorch:', torch.cuda.is_available())
"""

In [20]:
model = "mlabonne/NeuralBeagle14-7B"

In [21]:
messages = [
    {
        "role": "system",
        "content": "You are an informative programming assistant that understands the working of a function and replies with it's documentation in markdown syntax."
    },
    {
        "role": "user",
        "content": f"{function}"
    }
]

In [22]:
tokenizer = AutoTokenizer.from_pretrained(model)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16, # Remember that float32 - CPU and float16 - GPU, keep this in mind if RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.05s/it]


In [23]:
%%time
outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 